In [1]:
import numpy  as np

In [35]:
class Disease():
    def __init__(self, idx=-1, name=''):
        self.idx  = idx   # disease number
        self.name = name  # disease name
        self.mx   = []    # assoc. manifestations
        self.link = []    # linked diseases
        
    def __str__(self):
        return '{}: {} | # mx = {}, # link = {}'.format(self.idx, self.name,
                                                    len(self.mx), len(self.link))
    # get functions
    def get_idx(self):   return self.idx
    def get_name(self):  return self.name
    def get_mxs(self):   return self.mx
    def get_links(self): return self.link
    
    def print_mxs(self):
        for mx in self.mx:
            print(mx)
    
    # add functions
    def add_mx(self, new_mx):
        self.mx.append(new_mx)
        
    def add_link(self, new_link):
        self.link.append(new_link)
        
    # is_in functions
    def has_manifestation(self, m):
        return True if m in self.mx else False
    
    def has_link(self, l):
        return True if l in self.link else False 
    
    

In [36]:
class Manifestation():
    def __init__(self, idx=-1, name='', es=-1, fq=-1, im=-1):
        self.idx  = idx   # mx number
        self.name = name  # mx name
        self.es   = es    # evoking strength
        self.fq   = fq    # frequency
        self.im   = im    # import
        
    def __eq__(self, other):
        return self.idx == other.idx
    
    def __ne__(self, other):
        return not self == other
    
    def __str__(self):
        return '{}: {} | es = {}, fq = {}, im = {}'.format(self.idx, self.name,
                                                             self.es, self.fq, self.im)
        
    # nonlinear transformations
    def trans_es(self):
        trans = {0: 1, 1: 4, 2: 10, 3: 20, 4: 40, 5: 80}
        return trans[self.es]

    def trans_fq(self):
        trans = {1: -1, 2: -4, 3: -7, 4: -15, 5: -30}
        return trans[self.fq]

    def trans_im(self):
        trans = {1: -2, 2: -6, 3: -10, 4: -20, 5: -40}
        return trans[self.im]
        
    # get functions
    def get_idx(self):  return self.idx
    def get_name(self): return self.name
    def get_es(self):   return self.es
    def get_fq(self):   return self.fq
    def get_im(self):   return self.im
    
    def get_trans_es(self):   return trans_es(self.es)
    def get_trans_fq(self):   return trans_fq(self.fq)
    def get_trans_im(self):   return trans_im(self.im)
    
    # set functions
    def set_es(self, new_es):
        self.es = new_es
    def set_fq(self, new_fq):
        self.fq = new_fq
    def set_im(self, new_im):
        self.im = new_im

In [37]:
class Patient():
    def __init__(self, pos=None, neg=None):
        self.positive = pos  # + findings
        self.negative = neg  # - findings

    # get functions
    def get_positives(self): return self.positive
    def get_negatives(self): return self.negative
    
    # add functions
    def add_positive(self, new_pos):
        self.positive.append(new_pos)
        
    def add_negative(self, new_neg):
        self.negative.append(new_neg)

In [38]:
def parse_disease(line):
    return line[1], ' '.join(line[2:])

def parse_manifestation(line):
    return line[2], ' '.join(line[3:]), line[1][0], line[1][1]

def parse_link(line):
    return line[3], ' '.join(line[4:])

def parse_even(line):
    return line[1], ' '.join(line[2:])

def parse_odd(line):
    return line[0], line[2]

In [39]:
FINDINGS = {}
DISEASES = {}

In [40]:
# build master manifestations list from findings file (#, NAME, IM)
with open('./data/findings.txt', 'r') as fd_file:
    for i, line in enumerate(fd_file):
        line = line.strip('\n').split()
        if i % 2:
            idx, imp = parse_odd(line)
            FINDINGS[idx].set_im(imp)
        else:
            idx, name = parse_even(line)
            FINDINGS[idx] = Manifestation(idx, name)

In [41]:
# build set of diseases (#, NAME, MX, LINKS) and update manifestations (ES and FQ)
with open('./data/diseases.txt', 'r') as dz_file:
    for line in dz_file:
        if line != '\n':
            line = line.strip('\n').split()
            if line[0] == 'DX':
                dz_idx, name = parse_disease(line)
                DISEASES[dz_idx] = Disease(dz_idx, name)
            elif line[0] == 'MX':
                idx, name, es, fq = parse_manifestation(line)
                DISEASES[dz_idx].add_mx(Manifestation(idx, name, es, fq, FINDINGS[idx].get_im()))
            elif line[0] == 'LINK':
                idx, name = parse_link(line)
                DISEASES[dz_idx].add_link((idx, name))

In [42]:
# build patient object -- (+MX, -MX)
PATIENT = Patient([], [])

In [ ]:
# read in patient file
with open('./data/patient.txt', 'r') as pt_file:
    for line in pt_file:
        pass  # determine + or - finding; add finding to appropriate list

In [61]:
# build diagnosis hypothesis options (idx: score)
DDX = {}

In [62]:
# pull each possible dx_hyp from disease list (based on +MX)
for pos_finding in PATIENT.get_positives():
    for idx, dz in DISEASES.items():
        if dz.has_manifestation(pos_finding):
            DDX[idx] = None

In [ ]:
# calculate score for each dx_hyp

In [ ]:
# sort ddx[] in descending order
DDX.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# choose optimal ddx or return list in order